<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-scf196/blob/main/pivot_point_driven_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Psuedocode
1. **Import:** `numpy` for math, `matplotlib.pyplot` for plotting, `solve_ivp` for solving ODE, `FuncAnimation` for animating, `HTML` for displaying animation
2. **Define function:** ODE for motion of the pendulum with the addition of the moving pivot point:
$$ \ddot{\theta} + 2\gamma\dot{\theta} + \omega_0^2\sin(\theta) - \omega_d^2\sin(\theta-\omega_d t) = 0 $$
Functions:
* `pendulum_ode(t, y, g, l, gamma, omega_d)` for ODE
* `get_pivot_xy(t_plot, omega_d, x_p0, y_p0)` gives coordinates for pivot
* `get_pendulum_xy(t_plot, theta_plot, l)` gives coordinates for bob with respect to the pivot
* `get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum)` gives coordinates in the lab frame

3. **Solve:** Use `solve_ivp` to solve the ODE
5. **Animate:** Use `create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0)` to make an animation of the pivot & bob

## Import

In [124]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Define ODE

In [125]:
def pendulum_ode(t, y, g, l, gamma, omega_d):
  """
  Defines the ordinary differential equation for the pendulum motion.

  PARAMETERS:
  t : float
      Time.
  y : list
      State variable [theta, ang_vel].
  g : float
      Gravitational acceleration.
  l : float
      Pendulum length.
  gamma : float
      Damping coefficient.
  omega_d : float
      Driving frequency.

  RETURNS:
  dy_dt : list
      Derivative of the state variable.
  """
  theta, ang_vel = y # unpacking state variable
  dtheta_dt = ang_vel # angular velocity, theta_dot
  d2theta_dt2 = - 2 * gamma * ang_vel  - ( g / l ) ** 2 * np.sin(theta) + omega_d ** 2 * np.sin(theta - omega_d * t) # angular acceleration, theta_ddot
  dy_dt = [dtheta_dt, d2theta_dt2] # derivative of state variable, lists of ang_vels and accelerations
  return dy_dt

## Solve

In [126]:
# set parameters

# time parameters
t_min = 0
t_max = 10
t_span = [t_min, t_max] # time span for solve_ivp
t_plot = np.linspace(t_min, t_max, 1000) # generate array of times from min to max with 1000 values

# other parameters
g = l = 1 # natural frequency is sqrt(g/l), setting it to 1
gamma = 0.1 # damping coefficient
omega_d = 10 # driving frequency
theta_0 = 0 # initial angle at the start of the time span
ang_vel_0 = 0 # initial angular velocity
x_p0 = 0 # initial x-position of pivot
y_p0 = 0 # initial y-position of pivot

# initial condition
y_0 = [theta_0, ang_vel_0] # use the scalar initial angle and initial angular velocity

# solve differential equation
sol = solve_ivp(pendulum_ode,
                t_span,
                y_0,
                args=(g, l, gamma, omega_d),
                method='Radau',
                dense_output=True,
                )

theta_plot = sol.sol(t_plot)[0] # array of angles at different times

# define functions for coordinates

def get_pivot_xy(t_plot, omega_d, x_p0, y_p0):
  """
  Gives the x & y coordinates of the pivot point.

  PARAMETERS:
  t_plot : ndarray
      Time array.
  omega_d : float
      Driving frequency.
  x_p0 : float
      Initial x-coordinate of the pivot point.
  y_p0 : float
      Initial y-coordinate of the pivot point.

  RETURNS:
  [x_pivot, y_pivot] : list
      List of x & y coordinates of the pivot point.
  """
  x_pivot = x_p0 - l * np.sin(omega_d * t_plot)
  y_pivot = y_p0 + l * np.cos(omega_d * t_plot) - l
  return [x_pivot, y_pivot]

def get_pendulum_xy(t_plot, theta_plot, l):
  """
  Gives the x & y coordinates of the pendulum bob.

  PARAMETERS:
  t_plot : ndarray
      Time array.
  theta_plot : ndarray
      Angle array.
  l : float
      Pendulum length.

  RETURNS:
  [x_pendulum, y_pendulum] : list
      List of x & y coordinates of the pendulum bob.
  """
  x_pendulum = l * np.sin(theta_plot)
  y_pendulum = - l * np.cos(theta_plot)
  return [x_pendulum, y_pendulum]

def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum): # combines pivot_xy and pendulum_xy
  """
  Gives the x & y coordinates of the pendulum bob in the lab frame.

  PARAMETERS:
  x_pivot : ndarray
      x-positions of the pivot over time.
  y_pivot : ndarray
      y-positions of the pivot over time.
  x_pendulum : ndarray
      x-positions of the pendulum bob in the lab frame over time.
  y_pendulum : ndarray
      y-positions of the pendulum bob in the lab frame over time.

  RETURNS:
  [x_lab, y_lab] : list
      List of x & y coordinates of the pendulum bob in the lab frame.
  """
  x_lab = x_pivot + x_pendulum
  y_lab = y_pivot + y_pendulum
  return [x_lab, y_lab]

## Animation

In [123]:
# create pendulum animation
def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        toggle showing the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        factor to scale the animation speed (default is 1.0).
        a value > 1.0 slows down the animation, < 1.0 speeds it up.

    RETURNS:
    anim : FuncAnimation
        matplotlib animation object.
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

# animate motion
# plot pendulum angle versus time
# get coordinates for plotting
n_points = 300 # number of frames
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

# animation flags/parameters
show_traj = True # True --> show trajectory; False --> do not show trajectory
speed_factor = 1 # >1 --> slow down animation; <1 --> speed up animation

# create_pendulum_animation
pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
HTML(pendulum_animation.to_html5_video()) # display animation

## Takeaways

* Define ODE, solve to get a set of angles, use angles for the motion of the pivot point & animate

* Hard to configure animation when $\omega_d >> 10$, this should also be when the motion starts to get close to the pivot fully circling the pendulum bob

* $\omega_d >> γ$ leads to strange motion, when they are closer, looks more like small oscillations of the pendulum bob

* $\theta_0$ and $\dot{\theta}_0$ start at 0, but starting them elsewhere may impact the results by counteracting the force of the pivot point

* Much higher $\omega_d$ could stabilize the motion for good